In [3]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as pyplot
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython
from Utils.createURM import createURM

In [4]:
URM = createURM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createURM.py:24: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createICM.py:46: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [5]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2693 ( 6.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1524 ( 3.7%) Users that have less than 1 test interactions


## FunkSVD

In [10]:
recommender = MatrixFactorization_FunkSVD_Cython(URMICMCombined.T)

def objective(trial):
    num_factors = trial.suggest_float("num_factors", 50, 500)
    user_reg = trial.suggest_float("user_reg", 1, 10)
    item_reg = trial.suggest_float("item_reg", 1, 10)
    recommender.fit(
            epochs=10,
            batch_size=1000,
            num_factors=num_factors,
            positive_threshold_BPR=None,
            learning_rate=0.001,
            use_bias=False,
            sgd_mode='adam',
            negative_interactions_quota=0.0,
            init_mean=0.0, init_std_dev=0.1,
            user_reg=user_reg, item_reg=item_reg, bias_reg=0.0, positive_reg=0.0, negative_reg=0.0,
            random_seed=None,
            **{
                'epochs_min' : 0,
                'evaluator_object' : evaluator_validation,
                'stop_on_validation' : True,
                'validation_every_n' : 5,
                'validation_metric' : 'MAP',
                'lower_validations_allowed' : 3
            }
    )

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [ ]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2022-11-05 00:24:34,945] A new study created in memory with name: no-name-6accf0ff-5199-4869-a9b8-5839ff453437


FUNK_SVD: Processed 1099000 (100.0%) in 7.83 sec. MSE loss 9.87E-01. Sample per second: 140304
FUNK_SVD: Epoch 1 of 10. Elapsed time 7.70 sec
FUNK_SVD: Processed 1099000 (100.0%) in 8.51 sec. MSE loss 9.73E-01. Sample per second: 129165
FUNK_SVD: Epoch 2 of 10. Elapsed time 15.37 sec
FUNK_SVD: Processed 1099000 (100.0%) in 8.23 sec. MSE loss 9.69E-01. Sample per second: 133552
FUNK_SVD: Epoch 3 of 10. Elapsed time 23.09 sec
FUNK_SVD: Processed 1099000 (100.0%) in 7.88 sec. MSE loss 9.28E-01. Sample per second: 139496
FUNK_SVD: Epoch 4 of 10. Elapsed time 30.74 sec
FUNK_SVD: Processed 1099000 (100.0%) in 8.53 sec. MSE loss 8.21E-01. Sample per second: 128853
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 38936 (100.0%) in 24.41 sec. Users per second: 1595
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0000000, PRECISION_RECALL_MIN_DEN: 0.0000000, RECALL: 0.0000000, MAP: 0.0000000, MAP_MIN_DEN: 0.0000000, MRR: 0.0000000, NDCG: 0.0000000, F1: 0.0000000, HIT_RATE: 0.0000000, ARHR_ALL_HITS

[I 2022-11-05 00:27:04,269] Trial 0 finished with value: 0.0 and parameters: {'num_factors': 83.59486999144859, 'user_reg': 6.10542017857443, 'item_reg': 5.5045977638138}. Best is trial 0 with value: 0.0.


FUNK_SVD: Processed 1099000 (100.0%) in 13.36 sec. MSE loss 9.88E-01. Sample per second: 82247
FUNK_SVD: Epoch 1 of 10. Elapsed time 12.81 sec
FUNK_SVD: Processed 1099000 (100.0%) in 13.09 sec. MSE loss 9.89E-01. Sample per second: 83969
FUNK_SVD: Epoch 2 of 10. Elapsed time 25.53 sec
FUNK_SVD: Processed 1099000 (100.0%) in 12.78 sec. MSE loss 9.65E-01. Sample per second: 86019
FUNK_SVD: Epoch 3 of 10. Elapsed time 38.22 sec
FUNK_SVD: Processed 1099000 (100.0%) in 13.53 sec. MSE loss 9.41E-01. Sample per second: 81223
FUNK_SVD: Epoch 4 of 10. Elapsed time 50.98 sec
FUNK_SVD: Processed 1099000 (100.0%) in 13.25 sec. MSE loss 8.56E-01. Sample per second: 82922
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 38936 (100.0%) in 27.57 sec. Users per second: 1412
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0000000, PRECISION_RECALL_MIN_DEN: 0.0000000, RECALL: 0.0000000, MAP: 0.0000000, MAP_MIN_DEN: 0.0000000, MRR: 0.0000000, NDCG: 0.0000000, F1: 0.0000000, HIT_RATE: 0.0000000, ARHR_ALL_HIT

[I 2022-11-05 00:30:34,415] Trial 1 finished with value: 0.0 and parameters: {'num_factors': 141.3557010553778, 'user_reg': 7.72972693169171, 'item_reg': 9.960942930844908}. Best is trial 0 with value: 0.0.


FUNK_SVD: Processed 1099000 (100.0%) in 23.99 sec. MSE loss 9.52E-01. Sample per second: 45809
FUNK_SVD: Epoch 1 of 10. Elapsed time 23.01 sec
FUNK_SVD: Processed 1099000 (100.0%) in 23.79 sec. MSE loss 9.27E+01. Sample per second: 46197
FUNK_SVD: Epoch 2 of 10. Elapsed time 45.81 sec
FUNK_SVD: Processed 1099000 (100.0%) in 23.54 sec. MSE loss 7.46E-01. Sample per second: 46689
FUNK_SVD: Epoch 3 of 10. Elapsed time 1.14 min
FUNK_SVD: Processed 1099000 (100.0%) in 23.28 sec. MSE loss 5.69E-01. Sample per second: 47205
FUNK_SVD: Epoch 4 of 10. Elapsed time 1.52 min
FUNK_SVD: Processed 1099000 (100.0%) in 23.04 sec. MSE loss 1.34E+00. Sample per second: 47704
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 38936 (100.0%) in 28.54 sec. Users per second: 1364
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0000000, PRECISION_RECALL_MIN_DEN: 0.0000000, RECALL: 0.0000000, MAP: 0.0000000, MAP_MIN_DEN: 0.0000000, MRR: 0.0000000, NDCG: 0.0000000, F1: 0.0000000, HIT_RATE: 0.0000000, ARHR_ALL_HITS:

[I 2022-11-05 00:35:48,724] Trial 2 finished with value: 0.0 and parameters: {'num_factors': 263.86605136228616, 'user_reg': 2.3153588130118083, 'item_reg': 7.127435470073221}. Best is trial 0 with value: 0.0.


FUNK_SVD: Processed 1099000 (100.0%) in 34.19 sec. MSE loss 9.49E-01. Sample per second: 32143
FUNK_SVD: Epoch 1 of 10. Elapsed time 33.51 sec
FUNK_SVD: Processed 1099000 (100.0%) in 33.50 sec. MSE loss 8.96E-01. Sample per second: 32802
FUNK_SVD: Epoch 2 of 10. Elapsed time 1.11 min
FUNK_SVD: Processed 1099000 (100.0%) in 7.27 min. MSE loss 9.13E-01. Sample per second: 2518
FUNK_SVD: Epoch 3 of 10. Elapsed time 8.38 min
